# Facebook Messenger

This notebook shows how to load data from Facebook in a format you can finetune on.

First, download your messenger data following instructions [here](https://www.zapptales.com/en/download-facebook-messenger-chat-history-how-to/). IMPORTANT - make sure to download them in JSON format (not HTML).

In [1]:
path = "/Users/harrisonchase/Downloads/messages"

In [2]:
from langchain.chat_loaders.facebook_messenger import SingleFileFacebookMessengerChatLoader, FolderFacebookMessengerChatLoader

In [3]:
loader = SingleFileFacebookMessengerChatLoader(
    file_path="/Users/harrisonchase/Downloads/messages/inbox/deanshu_10159392635813226/message_1.json",
    person_name = "Harrison Chase"
)

In [4]:
messages = loader.load()

In [5]:
loader = FolderFacebookMessengerChatLoader(
    directory_path="/Users/harrisonchase/Downloads/messages",
    person_name = "Harrison Chase",
)

In [6]:
messages = loader.load()

In [7]:
from langchain.adapters.openai import convert_messages_for_finetuning

In [9]:
data = convert_messages_for_finetuning(messages)

In [10]:
data

[[{'role': 'user',
   'content': "hi Harrison! next week I'll mostly around 5th and Mission (in SOMA), let me know if there's any chance you'll be around there as well â\x80\x94 otherwise I can come find you if you'll be posted elsewhere"},
  {'role': 'user',
   'content': "mon/tues/fri are generally better days, lmk if there's particular times that work best for you"},
  {'role': 'assistant', 'content': 'Hey! Sorry for the late response'},
  {'role': 'assistant',
   'content': "You can now message and call each other and see info like Active Status and when you've read messages."},
  {'role': 'assistant',
   'content': 'Would it be possible to circle back on this in a few weeks? Things are a bit hectic right now, and Iâ\x80\x99m looking forward to chatting but need to prioritize a few other things that are pressing :)'},
  {'role': 'user', 'content': 'yeah of course man, take your time!'},
  {'role': 'user', 'content': "i'll check back in with you in 2 weeks"},
  {'role': 'user',
   '

In [11]:
import openai
import json
from io import BytesIO

my_file = BytesIO()
for m in data:
    my_file.write((json.dumps({"messages": m}) + "\n").encode('utf-8'))

my_file.seek(0)
training_file = openai.File.create(
  file=my_file,
  purpose='fine-tune'
)

In [18]:
training_file

<File file id=file-03vGq2Q9l136VxrvLB8iflQw at 0x12ca6c310> JSON: {
  "object": "file",
  "id": "file-03vGq2Q9l136VxrvLB8iflQw",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 105383,
  "created_at": 1692810257,
  "status": "uploaded",
  "status_details": null
}

In [3]:
job = openai.FineTuningJob.create(training_file="file-03vGq2Q9l136VxrvLB8iflQw", model="gpt-3.5-turbo")

In [4]:
import time
start = time.time()

while True:
    ftj = openai.FineTuningJob.retrieve(job.id)
    if ftj.fine_tuned_model is None:
        print(f"Waiting for fine-tuning to complete... Elapsed: {time.time() - start}", end="\r", flush=True)
        time.sleep(10)
    else:
        print(ftj.fine_tuned_model, flush=True)
        break

ft:gpt-3.5-turbo-0613:langchain::7qlkU9t4apsed: 577.73740625381474


In [8]:
ftj

<FineTuningJob fine_tuning.job id=ftjob-mj8dPg2on0dNVS8efn2yPxc6 at 0x103f08590> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-mj8dPg2on0dNVS8efn2yPxc6",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1692810357,
  "finished_at": 1692810961,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:langchain::7qlkU9t4",
  "organization_id": "org-i0zjYONU3PemzJ222esBaAzZ",
  "result_files": [
    "file-0mZu3g9fMaE0rxC5tgAjiVke"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-03vGq2Q9l136VxrvLB8iflQw",
  "hyperparameters": {
    "n_epochs": 4
  },
  "trained_tokens": 61792
}

In [16]:
from langchain.chat_models import ChatOpenAI
model = ChatOpenAI(temperature=0)
finetuned = ChatOpenAI(model="ft:gpt-3.5-turbo-0613:langchain::7qlkU9t4", temperature=0)

In [17]:
model.predict("hi! do you have time to chat?")

'Hello! Yes, I have time to chat. How can I assist you today?'

In [21]:
model.predict("what are you up to tonight?")

"As an AI, I don't have personal experiences or activities. I'm here to assist and provide information. How can I help you today?"

In [19]:
finetuned.predict("hi! do you have time to chat?")

"Of course! What's up?"

In [22]:
finetuned.predict("what are you up to tonight?")

"I'm just hanging out at home. What about you?"